In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

In [2]:
# Nicer plotting
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = (8,4)

# Disgten example

Similar to the simple example, but generating particles with Distgen

In [3]:
from distgen import Generator
YAML="""
beam:
  params:
    MTE:
      value: 414
      units: meV    
    total_charge:
      value: 250
      units: pC
      
  particle_count: 10000
  rand_type: hammersley
  start_type: cathode

r_dist:
  type: radial_uniform
  params:
    max_r:
      value: 0.6
      units: mm    


t_dist:
  type: tukey
  params:
    length:
      value: 7.5
      units: ps    
    ratio:  
      value: 0.8
      units: ''      

  
output:
  file: cu_inj.part
  type: astra  
"""

G = Generator(YAML)
G.run()
P = G.particles

In [4]:
from impact import Impact, template_dir

import matplotlib.pyplot as plt
import os

In [5]:
ifile = os.path.join(template_dir,'lcls_injector/ImpactT.in')
os.path.exists(ifile)

True

In [6]:
# Make Impact object
I = Impact(ifile, initial_particles = G.particles, verbose=True)

================ Impact-T Summary ================
1 bunchtotal charge: 249.99999999999997 pC
Distribution type: read
Cathode start at z = 0 m
   emission time: 1.4e-11 s
   image charges neglected after z = 0.02 m
Tracking 10000 particles
Processor domain: 1 x 1 = 1 CPUs
Computational domain: 0.015 m x 0.015 m x 45.0 m
Space charge grid: 16 x 16 x 16
Maximum time steps: 1000000
Random Seed: 6
Reference Frequency: 2856000000.0 Hz
Initial reference time: 0.0 s
Simulation starting from the beginning


Configured to run in: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmpe7bo2ufo


In [ ]:
# This will us
I.write_initial_particles(update_header=True)

In [ ]:
# Change some things
I.input['header']['Np'] = 10000
I.input['header']['Nx'] = 16
I.input['header']['Ny'] = 16
I.input['header']['Nz'] = 16
I.input['header']['Dt'] = 5e-13

# Other switches
I.timeout = 1000
# Switches for MPI
I.use_mpi=True
I.input['header']['Nprow'] = 1
I.input['header']['Npcol'] = 4
I.impact_bin = '$IMPACTT_MPI_BIN'
assert os.path.exists(os.path.expandvars(I.impact_bin))

In [ ]:
# Change stop location
I.ele['stop_1']['s'] = 0.1
#I.ele['stop_1']['s'] = I.ele['OTR2']['s']+.001

In [ ]:
I.run()

In [ ]:
I.input.keys()

In [ ]:
I.output.keys()

In [ ]:
I.output['stats'].keys()

In [ ]:
I.output['slice_info'].keys()

# Particles

In [ ]:
# Particles are automatically parsed in to openpmd-beamphysics ParticleGroup objects
I.output['particles']

In [ ]:
# Get the final particles, calculate some statistic
P = I.output['particles']['final_particles']
P['mean_energy']

In [ ]:
# Show the units
P.units('mean_energy')

In [ ]:
# This provides easy ploting
from pmd_beamphysics.plot import marginal_plot

marginal_plot(P, 'z', 'pz')

# Stats

In [ ]:
# Impact's own calculated statistics can be retieved
len(I.stat('norm_emit_x')), I.stat('norm_emit_x')[-1]

In [ ]:
# Stats can also be computed from the particles. For example:


In [ ]:
# Compare these. 
key1 = 'mean_z'
key2 = 'norm_emit_x'
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1+f' ({units1})')
plt.ylabel(key2+f' ({units2})')
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles], 
    [I.particles[name][key2] for name in I.particles], color='red')